In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely.prepared import prep
import matplotlib.pyplot as plt
import os
import rioxarray as rxr

In [3]:


dred = pd.read_csv('data/db_red_maple.csv', dtype={'latitude': 'float64', 'longitude': 'float64'})

In [4]:
dred

,Unnamed: 0,essence,latitude,forest,longitude,elev,ph,clay,sand,bio1,...,bio15,bio18,bio20,bio23,bio24,bio27,bio30,bio31,bio34,cluster_k
0,577458,EO,47.761208,100,-73.022446,344,4.5000,85.0,85.0,1.765863,...,24.643180,301.000,19.051277,78.625160,588.10000,35.535050,-6.559094,9.199162,733.57570,1
1,1033211,EO,46.432621,100,-75.445401,239,5.1067,13.0,51.0,3.338708,...,12.093941,294.750,17.747839,75.759040,625.57500,33.815228,-1.487061,8.546481,798.44330,3
2,911397,EO,46.028920,100,-74.745012,285,5.5000,2.0,93.0,2.942621,...,12.358034,304.150,16.637333,82.134315,653.25000,33.674984,-4.342159,6.658854,784.00290,3
3,69869,EO,46.839032,100,-70.519246,363,3.5167,24.0,45.0,3.264520,...,17.166555,343.700,15.861320,91.072400,721.94995,32.355210,-5.601314,10.016439,715.82350,0
4,1521793,EO,48.080635,91,-79.398379,331,5.7000,90.0,0.0,0.868738,...,24.928170,261.750,20.987915,76.240395,541.90000,36.527588,-11.209414,9.943170,717.10360,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847957,772824,EO,45.995362,100,-74.002996,372,6.1000,69.0,4.0,4.209070,...,9.217865,285.925,17.601393,72.901840,636.12500,32.952133,-0.846002,3.312908,781.30550,3
847958,1073892,EO,46.137758,100,-75.599060,328,4.6167,4.0,81.0,3.904474,...,12.178079,280.350,19.275383,68.300415,591.17500,33.885113,-0.613117,7.181046,802.66705,3
847959,1336875,EO,46.359258,100,-77.158444,318,4.5000,85.0,85.0,3.932580,...,16.052400,237.125,23.497679,59.750330,521.40000,34.233120,-2.733226,9.741150,812.64520,1
847960,403313,EO,47.485293,100,-68.537980,362,4.8000,8.0,34.0,2.571893,...,15.102576,275.700,18.190674,79.534220,590.47500,32.157380,-2.661742,8.380980,732.18720,3


In [5]:
# 2. Crear GeoDataFrame de presencias
geometry = [Point(lon, lat) for lon, lat in zip(dred['longitude'], dred['latitude'])]
presences_gdf = gpd.GeoDataFrame(dred.copy(), geometry=geometry, crs="EPSG:4326")

In [6]:
# 4. Reproyectar
from pyproj import Transformer
from shapely.ops import transform

In [7]:
print("Reproyectando presencias...")
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32198", always_xy=True)
presences_proj_geom = presences_gdf.geometry.apply(lambda geom: transform(transformer.transform, geom))
presences_proj = gpd.GeoDataFrame(presences_gdf.drop(columns='geometry'), geometry=presences_proj_geom)
presences_proj.set_crs("EPSG:32198", allow_override=True, inplace=True)

Reproyectando presencias...


,Unnamed: 0,essence,latitude,forest,longitude,elev,ph,clay,sand,bio1,...,bio18,bio20,bio23,bio24,bio27,bio30,bio31,bio34,cluster_k,geometry
0,577458,EO,47.761208,100,-73.022446,344,4.5000,85.0,85.0,1.765863,...,301.000,19.051277,78.625160,588.10000,35.535050,-6.559094,9.199162,733.57570,1,POINT (-337745.54 428899.35)
1,1033211,EO,46.432621,100,-75.445401,239,5.1067,13.0,51.0,3.338708,...,294.750,17.747839,75.759040,625.57500,33.815228,-1.487061,8.546481,798.44330,3,POINT (-532511.46 296685.91)
2,911397,EO,46.028920,100,-74.745012,285,5.5000,2.0,93.0,2.942621,...,304.150,16.637333,82.134315,653.25000,33.674984,-4.342159,6.658854,784.00290,3,POINT (-482865.26 247053.66)
3,69869,EO,46.839032,100,-70.519246,363,3.5167,24.0,45.0,3.264520,...,343.700,15.861320,91.072400,721.94995,32.355210,-5.601314,10.016439,715.82350,0,POINT (-153767.5 318115.51)
4,1521793,EO,48.080635,91,-79.398379,331,5.7000,90.0,0.0,0.868738,...,261.750,20.987915,76.240395,541.90000,36.527588,-11.209414,9.943170,717.10360,1,POINT (-805940.7 515109.13)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847957,772824,EO,45.995362,100,-74.002996,372,6.1000,69.0,4.0,4.209070,...,285.925,17.601393,72.901840,636.12500,32.952133,-0.846002,3.312908,781.30550,3,POINT (-425899.81 238624.65)
847958,1073892,EO,46.137758,100,-75.599060,328,4.6167,4.0,81.0,3.904474,...,280.350,19.275383,68.300415,591.17500,33.885113,-0.613117,7.181046,802.66705,3,POINT (-547498.74 265245.62)
847959,1336875,EO,46.359258,100,-77.158444,318,4.5000,85.0,85.0,3.932580,...,237.125,23.497679,59.750330,521.40000,34.233120,-2.733226,9.741150,812.64520,1,POINT (-664258.1 302910.31)
847960,403313,EO,47.485293,100,-68.537980,362,4.8000,8.0,34.0,2.571893,...,275.700,18.190674,79.534220,590.47500,32.157380,-2.661742,8.380980,732.18720,3,POINT (-2854.52 387637.46)
